In [1]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy.timeseries import TimeSeries
import pandas as pd
import os
import glob
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('astro')
from astropy.io import fits
from uncertainties import ufloat
from uncertainties.umath import sqrt

In [2]:
# Base directory where 'folded_curves' is located
nicer_base_dir = '/home/supremekai/thesis/J0243_6p6124/NICER_second_run/folded_curves'

# Find all NICER folded curve FITS files recursively
all_nicer_fits_files = glob.glob(
    #os.path.join(nicer_base_dir, '**', '*_rms_deviation_flux_errors.fits'),
    os.path.join(nicer_base_dir,'**','*_pulse_profile_lc_files_efold_output_same_epoch.fits'),
    recursive=True
)

# Pair each file with its obsID extracted from its parent directory name
nicer_files_with_obsid = [(f, os.path.basename(os.path.dirname(f))) for f in all_nicer_fits_files]

# Sort based on the numeric value of obsID
nicer_files_with_obsid_sorted = sorted(nicer_files_with_obsid, key=lambda x: int(x[1]))

# Unpack into separate lists
nicer_files = [f for f, _ in nicer_files_with_obsid_sorted]
nicer_obs_ids = [obs for _, obs in nicer_files_with_obsid_sorted]

# Output some info for inspection
print(f"Total NICER FITS files found: {len(nicer_files)}")
print("First few (sorted) NICER files with ObsIDs:")
for f, obs in nicer_files_with_obsid_sorted[:5]:
    print(f"ObsID: {obs}, File: {f}")

Total NICER FITS files found: 43
First few (sorted) NICER files with ObsIDs:
ObsID: 6050390227, File: /home/supremekai/thesis/J0243_6p6124/NICER_second_run/folded_curves/6050390227/6050390227_pulse_profile_lc_files_efold_output_same_epoch.fits
ObsID: 6050390228, File: /home/supremekai/thesis/J0243_6p6124/NICER_second_run/folded_curves/6050390228/6050390228_pulse_profile_lc_files_efold_output_same_epoch.fits
ObsID: 6050390229, File: /home/supremekai/thesis/J0243_6p6124/NICER_second_run/folded_curves/6050390229/6050390229_pulse_profile_lc_files_efold_output_same_epoch.fits
ObsID: 6050390230, File: /home/supremekai/thesis/J0243_6p6124/NICER_second_run/folded_curves/6050390230/6050390230_pulse_profile_lc_files_efold_output_same_epoch.fits
ObsID: 6050390231, File: /home/supremekai/thesis/J0243_6p6124/NICER_second_run/folded_curves/6050390231/6050390231_pulse_profile_lc_files_efold_output_same_epoch.fits


In [3]:
def read_files(path):
    profile = fits.open(path)
    data = profile[1].data
    binned_rate = data['RATE1']
    phase = data['PHASE']
    binned_rate_error = data['ERROR1']
    
    N = len(binned_rate)
    r_i = [ufloat(binned_rate[i], binned_rate_error[i]) for i in range(N)]

    I_max = max(r_i)
    I_min = min(r_i)
    pulsed_fraction = (I_max - I_min) / (I_max + I_min)

    r_bar = sum(r_i) / N
    numerator = sqrt(sum((r - r_bar) ** 2 for r in r_i) / N)
    rms_pulsed_fraction = numerator / r_bar

    return phase, binned_rate, binned_rate_error, pulsed_fraction, rms_pulsed_fraction


In [4]:
def process_nicer_group(nicer_files_with_obsid_sorted, save_dir='./pf_results', plot=False):
    """Calculate and save pulsed fractions for NICER pulse profile FITS files."""
    pf_data = []
    os.makedirs(save_dir, exist_ok=True)

    for file_path, obsid in nicer_files_with_obsid_sorted:
        try:
            phase, intensity, error, pf, rms_pf = read_files(file_path)

            pf_data.append({
                "ObsID": obsid,
                "FilePath": file_path,
                "PulsedFraction": round(pf.nominal_value, 5),
                "PulsedFraction_Error": round(pf.std_dev, 5),
                "RMS_PulsedFraction": round(rms_pf.nominal_value, 5),
                "RMS_PulsedFraction_Error": round(rms_pf.std_dev, 5)
            })

        except Exception as e:
            print(f"Error processing {file_path}: {e}")

    # Save results
    df = pd.DataFrame(pf_data)
    csv_path = os.path.join(save_dir, 'NICER_pulsed_fractions.csv')
    df.to_csv(csv_path, index=False)
    print(f"Saved NICER pulsed fraction results to: {csv_path}")


In [5]:
def main():
    process_nicer_group(nicer_files_with_obsid_sorted)

if __name__ == '__main__':
    main()


Saved NICER pulsed fraction results to: ./pf_results/NICER_pulsed_fractions.csv
